In [60]:
from binance.client import Client
import pandas as pd
from time import sleep

In [55]:
api = "KQ0WSoJWjc9jccInXJPi7xZbKV2VA024oLgZx1kFPHdRcbw1nTn6NpUArL4SFT1M"
secret = "PPHApTk4jaMK4rP3LKNLHpJ6UvyRjhHHemS7QvhUnneSMgA9CKGHlty40O5u3ynz"

In [56]:
client = Client(api, secret)

In [74]:
def get_hour_data(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + " hour ago UTC"))
    frame = frame.iloc[:, 0:6]
    frame.columns = ["time", "open", "high", "low", "close", "volume"]
    frame = frame.set_index("time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [75]:
get_hour_data('BTCUSDT', '1h', '3')

,open,high,low,close,volume,ret,price
time,,,,,,,
2024-08-15 06:00:00,58315.01,58390.00,57847.32,58101.44,1550.94234,NaN,58101.44
2024-08-15 07:00:00,58101.44,58149.00,57713.88,58060.37,1156.56805,-0.000707,58060.36
2024-08-15 08:00:00,58060.36,58335.33,57964.85,58313.48,851.07714,0.004359,NaN


In [59]:
def strategytest(symbol, qty, entried=False):
    df = get_hour_data(symbol, '1h', '3')
    df['ret'] = df['close'].pct_change()
    df['price'] = df['open'].shift(-1)
    in_position = entried

    if not in_position:
        # Check entry condition
        if df['ret'].iloc[-1] > 0.01:
            buy_price = df['price'].iloc[-1]
            tp = buy_price * 1.05
            sl = buy_price * 0.95
            order = client.create_order(symbol=symbol, side='BUY', type='MARKET', quantity=qty)
            print(f"Entry order executed: {order}")
            print(order)
            in_position = True
            buy_time = pd.to_datetime(order['transactTime'], unit='ms')
    else:
        while in_position:
            df = get_hour_data(symbol, '1h', '3')
            since_buy = df.loc[df.index > buy_time]
            if len(since_buy) > 0:
                for _, row in since_buy.iterrows():
                    if row['high'] > tp:
                        order = client.create_order(symbol=symbol, side='SELL', type='MARKET', quantity=qty)
                        print(f"Take Profit order executed: {order}")
                        in_position = False
                        break
                    elif row['low'] < sl:
                        order = client.create_order(symbol=symbol, side='SELL', type='MARKET', quantity=qty)
                        print(f"Stop Loss order executed: {order}")
                        in_position = False
                        break
            sleep(3600)  # Wait for 1 hour before the next check

In [76]:
def run_strategy_continuously(symbol, qty):
    entried = False
    while True:
        strategytest(symbol, qty, entried)
        sleep(3600)

run_strategy_continuously('BTCUSDT', 0.001)
